# Introduction
This is details of a VR-PIPE setup of full calling pipeline of 100bp reads from 16 "multiple reference strain" (MRS) samples. I originally tried doing this without success from Thomas's mapping (see 20160523_pf3k_mrs_1_setup.ipynb). I then tried to set up using a vrtrack database, but this failed, we think becuase of problems with changes to iRODs commands (and also an issue with libs table in solaris that Cinzia is looking into). Trying here by manually downloading files from iRODs and creating a fofn.

In [1]:
%run _standard_imports.ipynb
%run _shared_setup.ipynb


python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
RELEASE_DIR = "%s/mrs_1" % DATA_DIR
RESOURCES_DIR = '%s/resources' % RELEASE_DIR

# GENOME_FN = "/nfs/pathogen003/tdo/Pfalciparum/3D7/Reference/Oct2011/Pf3D7_v3.fasta" # Note this ref used by Thomas is different to other refs we have used, e.g. chromsomes aren't in numerical order
GENOME_FN = "/lustre/scratch109/malaria/pf3k_methods/resources/Pfalciparum.genome.fasta"
SNPEFF_DIR = "/lustre/scratch109/malaria/pf3k_methods/resources/snpEff"
REGIONS_FN = "/nfs/team112_internal/rp7/src/github/malariagen/pf-crosses/meta/regions-20130225.bed.gz"

RELEASE_METADATA_FN = "%s/pf3k_mrs_1_sample_metadata.txt" % RELEASE_DIR
WG_VCF_FN = "%s/vcf/pf3k_mrs_1.vcf.gz" % RELEASE_DIR

BCFTOOLS = '/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/pf3k_techbm/opt_4/bcftools/bcftools'

VRPIPE_FOFN = "%s/pf3k_mrs_1.fofn" % RELEASE_DIR

In [3]:
print(WG_VCF_FN)

/nfs/team112_internal/rp7/data/pf3k/mrs_1/vcf/pf3k_mrs_1.vcf.gz


In [4]:
chromosomes = ["Pf3D7_%02d_v3" % x for x in range(1, 15, 1)] + [
    'Pf3D7_API_v3', 'Pf_M76611'
]
chromosome_vcfs = ["%s/vcf/SNP_INDEL_%s.combined.filtered.vcf.gz" % (RELEASE_DIR, x) for x in chromosomes]


In [5]:
if not os.path.exists(RESOURCES_DIR):
    os.makedirs(RESOURCES_DIR)

In [6]:
!cp {GENOME_FN}* {RESOURCES_DIR}
!cp -R {SNPEFF_DIR} {RESOURCES_DIR}
!cp -R {REGIONS_FN} {RESOURCES_DIR}

In [7]:
for lustre_dir in ['input', 'output', 'meta']:
    os.makedirs("/lustre/scratch109/malaria/pf3k_mrs_1/%s" % lustre_dir)

FileExistsError: [Errno 17] File exists: '/lustre/scratch109/malaria/pf3k_mrs_1/input'

# Manual import

In [6]:
VRPIPE_FOFN

'/nfs/team112_internal/rp7/data/pf3k/mrs_1/pf3k_mrs_1.fofn'

In [20]:
# Download files from iRODS
for rec in etl.fromtsv(VRPIPE_FOFN).data():
    output_fn = rec[0]
    irods_fn = '/seq/19406/%s' % os.path.basename(output_fn)
    print(output_fn, irods_fn)
    !/software/irods-3.3/icommands/bin/iget {irods_fn} {output_fn}

/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#1.cram /seq/19406/19406_1#1.cram
ERROR: getUtil: get error for /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#1.cram status = -312000 OVERWRITE_WITHOUT_FORCE_FLAG
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#2.cram /seq/19406/19406_1#2.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#3.cram /seq/19406/19406_1#3.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#4.cram /seq/19406/19406_1#4.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#5.cram /seq/19406/19406_1#5.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#6.cram /seq/19406/19406_1#6.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#7.cram /seq/19406/19406_1#7.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#8.cram /seq/19406/19406_1#8.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_2#1.cram /seq/19406/19406_2#1.cram
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_2#2.cram /seq/19406/19406_2#2.cram
/lustre/scratch

In [21]:
for rec in etl.fromtsv(VRPIPE_FOFN).data():
    input_fn = rec[0]
    output_fn = rec[0].replace('cram', 'bam')
    print(input_fn, output_fn)
    !/software/solexa/pkg/samtools/samtools-1.2/samtools view -Sb -F 2816 {input_fn} -o {output_fn}

/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#1.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#1.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#2.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#2.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#3.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#3.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#4.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#4.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#5.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#5.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#6.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#6.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#7.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#7.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#8.cram /lustre/scratch109/malaria/pf3k_mrs_1/input/19406_1#8.bam
/lustre/scratch109/malaria/pf3k_mrs_1/input/19406_2#1.cram /lust

# VR-PIPE pipeline
These were created using vrpipe-setup --based_on, with the setups originally created in 20160523_pf3k_mrs_1_setup

# Create whole genome VCF

In [10]:
!{BCFTOOLS} concat {" ".join(chromosome_vcfs)} \
| sed 's/##FORMAT=<ID=AD,Number=./##FORMAT=<ID=AD,Number=R/' \
| bgzip -c > {WG_VCF_FN}
!tabix -p vcf {WG_VCF_FN}

In [11]:
WG_VCF_FN

'/nfs/team112_internal/rp7/data/pf3k/mrs_1/vcf/pf3k_mrs_1.vcf.gz'

# Create bam summary file

In [12]:
RELEASE_METADATA_FN

'/nfs/team112_internal/rp7/data/pf3k/mrs_1/pf3k_mrs_1_sample_metadata.txt'

In [18]:
!vrpipe-fileinfo \
--setup pf3k_mrs_1_bqsr \
--step gatk_print_reads_gatk3 \
--metadata sample,study,bases_callable_percent,bases_no_coverage_percent,bases_low_coverage_percent,bases_poor_mapping_quality_percent,mean_coverage \
| grep -v '\.bai' \
> {RELEASE_METADATA_FN}


# Create README section containing variant numbers

In [19]:
number_of_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'TYPE="snp"' {WG_VCF_FN} | wc -l
number_of_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'TYPE!="snp"' {WG_VCF_FN} | wc -l
number_of_pass_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp"' {WG_VCF_FN} | wc -l
number_of_pass_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp"' {WG_VCF_FN} | wc -l
number_of_pass_biallelic_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp" && N_ALT=1' {WG_VCF_FN} | wc -l
number_of_pass_biallelic_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp" && N_ALT=1' {WG_VCF_FN} | wc -l
number_of_VQSLODgt6_snps = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE="snp" && VQSLOD>6' {WG_VCF_FN} | wc -l
number_of_VQSLODgt6_indels = !{BCFTOOLS} query -f '%CHROM\t%POS\n' --include 'FILTER="PASS" && TYPE!="snp" && VQSLOD>6' {WG_VCF_FN} | wc -l

print("%s variants" % ("{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0]))))
print("%s SNPs" % ("{:,}".format(int(number_of_snps[0]))))
print("%s indels" % ("{:,}".format(int(number_of_indels[0]))))
print()
print("%s PASS variants" % ("{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0]))))
print("%s PASS SNPs" % ("{:,}".format(int(number_of_pass_snps[0]))))
print("%s PASS indels" % ("{:,}".format(int(number_of_pass_indels[0]))))
print()
print("%s PASS biallelic variants" % ("{:,}".format(int(number_of_pass_biallelic_snps[0]) + int(number_of_pass_biallelic_indels[0]))))
print("%s PASS biallelic SNPs" % ("{:,}".format(int(number_of_pass_biallelic_snps[0]))))
print("%s PASS biallelic indels" % ("{:,}".format(int(number_of_pass_biallelic_indels[0]))))
print()
print("%s VQSLOD>6.0 variants" % ("{:,}".format(int(number_of_VQSLODgt6_snps[0]) + int(number_of_VQSLODgt6_indels[0]))))
print("%s VQSLOD>6.0 SNPs" % ("{:,}".format(int(number_of_VQSLODgt6_snps[0]))))
print("%s VQSLOD>6.0 indels" % ("{:,}".format(int(number_of_VQSLODgt6_indels[0]))))
print()

574,132 variants
358,702 SNPs
215,430 indels

168,486 PASS variants
73,829 PASS SNPs
94,657 PASS indels

111,694 PASS biallelic variants
66,073 PASS biallelic SNPs
45,621 PASS biallelic indels

78,657 VQSLOD>6.0 variants
41,352 VQSLOD>6.0 SNPs
37,305 VQSLOD>6.0 indels



In [20]:
print('''
The VCF file contains details of {number_of_variants} discovered variants of which {number_of_snps}
are SNPs and {number_of_indels} are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are {number_of_pass_variants} such high-
quality PASS variants ({number_of_pass_snps} SNPs and {number_of_pass_indels} indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a quality score called VQSLOD. All
variants with a VQSLOD score below 0 are filtered out, i.e. will have a value of
Low_VQSLOD in the FILTER column, rather than PASS. The VQSLOD score for each
variant can be found in the INFO field of the VCF file. It is possible to use the
VQSLOD score to define a more or less stringent set of variants. For example for
a very stringent set of the highest quality variants, select only those variants
where VQSLOD >= 6. There are {number_of_VQSLODgt6_snps} such stringent SNPs and {number_of_VQSLODgt6_indels}
such stringent indels.

It is also important to note that some variants have more than two alleles. For
example, amongst the {number_of_pass_snps} high quality PASS SNPs, {number_of_pass_biallelic_snps} are biallelic. The
remaining {number_of_pass_multiallelic_snps} high quality PASS SNPs have 3 or more alleles. Similarly, amongst
the {number_of_pass_indels} high-quality PASS indels, {number_of_pass_biallelic_indels} are biallelic. The remaining
{number_of_pass_multiallelic_indels} high quality PASS indels have 3 or more alleles.
'''.format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_snps="{:,}".format(int(number_of_snps[0])),
        number_of_indels="{:,}".format(int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        number_of_pass_snps="{:,}".format(int(number_of_pass_snps[0])),
        number_of_pass_indels="{:,}".format(int(number_of_pass_indels[0])),
        number_of_VQSLODgt6_snps="{:,}".format(int(number_of_VQSLODgt6_snps[0])),
        number_of_VQSLODgt6_indels="{:,}".format(int(number_of_VQSLODgt6_indels[0])),
        number_of_pass_biallelic_snps="{:,}".format(int(number_of_pass_biallelic_snps[0])),
        number_of_pass_biallelic_indels="{:,}".format(int(number_of_pass_biallelic_indels[0])),
        number_of_pass_multiallelic_snps="{:,}".format(int(number_of_pass_snps[0]) - int(number_of_pass_biallelic_snps[0])),
        number_of_pass_multiallelic_indels="{:,}".format(int(number_of_pass_indels[0]) - int(number_of_pass_biallelic_indels[0])),
    )
)


The VCF file contains details of 574,132 discovered variants of which 358,702
are SNPs and 215,430 are indels (or multi-allelic mixtures of SNPs
and indels). It is important to note that many of these variants are
considered low quality. Only the variants for which the FILTER column is set
to PASS should be considered of high quality. There are 168,486 such high-
quality PASS variants (73,829 SNPs and 94,657 indels).

The FILTER column is based on two types of information. Firstly certain regions
of the genome are considered "non-core". This includes sub-telomeric regions,
centromeres and internal VAR gene clusters on chromosomes 4, 6, 7, 8 and 12.
The apicoplast and mitochondrion are also considered non-core. All variants within
non-core regions are considered to be low quality, and hence will not have the
FILTER column set to PASS. The regions which are core and non-core can be found
in the file resources/regions-20130225.bed.gz.

Secondly, variants are filtered out based on a quali

In [21]:
"{number_of_pass_variants}/{number_of_variants} variants ({pct_pass}%) pass all filters".format(
        number_of_variants="{:,}".format(int(number_of_snps[0]) + int(number_of_indels[0])),
        number_of_pass_variants="{:,}".format(int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])),
        pct_pass=round((
            (int(number_of_pass_snps[0]) + int(number_of_pass_indels[0])) /
            (int(number_of_snps[0]) + int(number_of_indels[0]))
        ) * 100)
)

'168,486/574,132 variants (29%) pass all filters'

# Make all files read-only

In [23]:
!chmod -R uga-w {RELEASE_DIR}

chmod: changing permissions of `/nfs/team112_internal/rp7/data/pf3k/mrs_1/pf3k_mrs_1_thomas_bams.txt': Operation not permitted
